# Kepler Framework, Emulate v13

In [1]:
from kepler.pandas.menu       import ElectronSequence as Chain
from kepler.pandas.readers    import load, load_in_loop, load_hdf
from core import Efficiency
from Gaugi import mkdir_p, progressbar
import numpy as np
import pandas as pd
import collections
import os
from pprint import pprint
from copy import deepcopy

import gc
%config InlineBackend.figure_format = 'retina'
%load_ext autoreload
%autoreload 2

Welcome to JupyROOT 6.16/00
Using all sub packages with ROOT dependence
INFO: Pandarallel will run on 40 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.

Applying ATLAS style settings...

Applying ATLAS style settings...

Applying ATLAS style settings...


In [2]:
output_path = 'output/emulation'
mkdir_p(output_path)

In [8]:
dpath_data17 = '../../data/output/data17_13TeV.AllPeriods.sgn.probes_lhvloose_EGAM1.bkg.vprobes_vlhvloose_EGAM7.GRL_v97.dataframe.h5'
dpath_data18 = '../../data/output/data18_13TeV.AllPeriods.sgn.probes_lhvloose_EGAM1.bkg.vprobes_vlhvloose_EGAM7.GRL_v97.dataframe.h5'

In [9]:
tpath_data17 = 'output/emulation/data17_table_v13.h5'
tpath_data18 = 'output/emulation/data18_table_v13.h5'

## Setup Chains:

In [10]:
# create my chain
chains = [
            Chain( "HLT_e17_lhvloose_nod0_ringer_v13_L1EM15VHI", L1Seed = 'L1_EM15VHI' , l2calo_column = 'ringer_v13_vloose'),
            Chain( "HLT_e28_lhtight_nod0_ringer_v13_ivarloose", L1Seed = 'L1_EM22VHI' , l2calo_column = 'ringer_v13_tight' ),
            Chain( "HLT_e60_lhmedium_nod0_ringer_v13_L1EM24VHI", L1Seed = 'L1_EM24VHI' , l2calo_column = 'ringer_v13_medium'),
            Chain( "HLT_e140_lhloose_nod0_ringer_v13" , L1Seed = 'L1_EM24VHI', l2calo_column = 'ringer_v13_loose'),
]

def emulate( df ):
    for chain in progressbar(chains, prefix='Emulate...'):
        chain.apply(df)

OrderedDict([('L1Seed', 'L1_EM15VHI'),
             ('L2Calo', (14000.0, 'ringer_v13_vloose')),
             ('L2', 'trig_L2_el_cut_pt15to20'),
             ('EFCalo', 17000.0),
             ('HLT', (17000.0, 'trig_EF_el_lhvloose'))])
OrderedDict([('L1Seed', 'L1_EM22VHI'),
             ('L2Calo', (25000.0, 'ringer_v13_tight')),
             ('L2', 'trig_L2_el_cut_pt20to50'),
             ('EFCalo', 28000.0),
             ('HLT', (28000.0, 'trig_EF_el_lhtight_ivarloose'))])
OrderedDict([('L1Seed', 'L1_EM24VHI'),
             ('L2Calo', (57000.0, 'ringer_v13_medium')),
             ('L2', 'trig_L2_el_cut_pt50toInf'),
             ('EFCalo', 60000.0),
             ('HLT', (60000.0, 'trig_EF_el_lhmedium'))])
OrderedDict([('L1Seed', 'L1_EM24VHI'),
             ('L2Calo', (137000.0, 'ringer_v13_loose')),
             ('L2', 'trig_L2_el_cut_pt50toInf'),
             ('EFCalo', 140000.0),
             ('HLT', (140000.0, 'trig_EF_el_lhloose'))])


## Emulate 2017:

Take something like 1 hour

In [11]:
data17_df = pd.concat( (load_hdf(dpath_data17), load_hdf(tpath_data17)), axis=1)

In [12]:
emulate(data17_df)

Emulate...: 100%|██████████| 4/4 [01:32<00:00, 23.18s/it]


In [13]:
data17_df.shape

(43311283, 112)

### Validate:

In [14]:
eff_data17_Zee = Efficiency( output_path+'/data17_Zee_efficiency_v13.root')

2022-03-24 09:55:55,532 | Py.Efficiency                           INFO Creating the StoreGate service with path: output/emulation/data17_Zee_efficiency_v13.root


In [15]:
eff_data17_Zee.fill( data17_df.loc[data17_df['target']==1], 'HLT_e17_lhvloose_nod0_ringer_v13_L1EM15VHI', pidname='el_lhvloose', reset=True)
eff_data17_Zee.fill( data17_df.loc[data17_df['target']==1], 'HLT_e28_lhtight_nod0_ringer_v13_ivarloose' , pidname='el_lhtight' , reset=True)
eff_data17_Zee.fill( data17_df.loc[data17_df['target']==1], 'HLT_e60_lhmedium_nod0_ringer_v13_L1EM24VHI', pidname='el_lhmedium', reset=True)
eff_data17_Zee.fill( data17_df.loc[data17_df['target']==1], 'HLT_e140_lhloose_nod0_ringer_v13'          , pidname='el_lhloose' , reset=True)

Filling...: 100%|██████████| 5/5 [01:53<00:00, 22.73s/it]


HLT_e17_lhvloose_nod0_ringer_v13_L1EM15VHI
+--------+----------+-------------------+
|  Step  | Eff [%%] |    passed/total   |
+--------+----------+-------------------+
| L1Calo | 94.1069  | 30158981/32047563 |
| L2Calo | 93.4395  | 29945080/32047563 |
|   L2   | 93.2021  | 29869012/32047563 |
| EFCalo | 91.8918  | 29449076/32047563 |
|  HLT   | 89.9880  | 28838954/32047563 |
+--------+----------+-------------------+


Filling...: 100%|██████████| 5/5 [01:26<00:00, 17.36s/it]


HLT_e28_lhtight_nod0_ringer_v13_ivarloose
+--------+----------+-------------------+
|  Step  | Eff [%%] |    passed/total   |
+--------+----------+-------------------+
| L1Calo | 94.6937  | 21557594/22765602 |
| L2Calo | 94.2397  | 21454224/22765602 |
|   L2   | 94.1247  | 21428044/22765602 |
| EFCalo | 93.2008  | 21217728/22765602 |
|  HLT   | 87.0402  | 19815231/22765602 |
+--------+----------+-------------------+


Filling...: 100%|██████████| 5/5 [00:05<00:00,  1.19s/it]


HLT_e60_lhmedium_nod0_ringer_v13_L1EM24VHI
+--------+----------+-----------------+
|  Step  | Eff [%%] |   passed/total  |
+--------+----------+-----------------+
| L1Calo | 99.5079  | 1361055/1367786 |
| L2Calo | 97.2700  | 1330446/1367786 |
|   L2   | 97.1803  | 1329218/1367786 |
| EFCalo | 95.0879  | 1300599/1367786 |
|  HLT   | 91.8108  | 1255775/1367786 |
+--------+----------+-----------------+


Filling...: 100%|██████████| 5/5 [00:00<00:00, 20.09it/s]

HLT_e140_lhloose_nod0_ringer_v13
+--------+----------+--------------+
|  Step  | Eff [%%] | passed/total |
+--------+----------+--------------+
| L1Calo | 100.0000 | 60541/60541  |
| L2Calo | 97.8956  | 59267/60541  |
|   L2   | 97.8296  | 59227/60541  |
| EFCalo | 96.5098  | 58428/60541  |
|  HLT   | 94.5739  | 57256/60541  |
+--------+----------+--------------+


In [16]:
eff_data17_Zee.save()

In [17]:
eff_data17_JF17 = Efficiency( output_path+'/data17_JF17_efficiency_v13.root')

2022-03-24 10:00:21,993 | Py.Efficiency                           INFO Creating the StoreGate service with path: output/emulation/data17_JF17_efficiency_v13.root


In [18]:
eff_data17_JF17.fill( data17_df.loc[data17_df['target']==0], 'HLT_e17_lhvloose_nod0_ringer_v13_L1EM15VHI', pidname='!el_lhvloose', reset=True)
eff_data17_JF17.fill( data17_df.loc[data17_df['target']==0], 'HLT_e28_lhtight_nod0_ringer_v13_ivarloose' , pidname='!el_lhvloose', reset=True)
eff_data17_JF17.fill( data17_df.loc[data17_df['target']==0], 'HLT_e60_lhmedium_nod0_ringer_v13_L1EM24VHI', pidname='!el_lhvloose', reset=True)
eff_data17_JF17.fill( data17_df.loc[data17_df['target']==0], 'HLT_e140_lhloose_nod0_ringer_v13'          , pidname='!el_lhvloose', reset=True)

Filling...: 100%|██████████| 5/5 [00:21<00:00,  4.36s/it]


HLT_e17_lhvloose_nod0_ringer_v13_L1EM15VHI
+--------+----------+------------------+
|  Step  | Eff [%%] |   passed/total   |
+--------+----------+------------------+
| L1Calo | 84.4275  | 8738241/10349992 |
| L2Calo |  1.6083  | 166455/10349992  |
|   L2   |  1.5509  | 160522/10349992  |
| EFCalo |  1.3762  | 142440/10349992  |
|  HLT   |  0.1416  |  14655/10349992  |
+--------+----------+------------------+


Filling...: 100%|██████████| 5/5 [00:19<00:00,  3.91s/it]


HLT_e28_lhtight_nod0_ringer_v13_ivarloose
+--------+----------+-----------------+
|  Step  | Eff [%%] |   passed/total  |
+--------+----------+-----------------+
| L1Calo | 90.6060  | 8384948/9254300 |
| L2Calo |  0.6214  |  57504/9254300  |
|   L2   |  0.5835  |  53999/9254300  |
| EFCalo |  0.4700  |  43494/9254300  |
|  HLT   |  0.0016  |   147/9254300   |
+--------+----------+-----------------+


Filling...: 100%|██████████| 5/5 [00:16<00:00,  3.31s/it]


HLT_e60_lhmedium_nod0_ringer_v13_L1EM24VHI
+--------+----------+-----------------+
|  Step  | Eff [%%] |   passed/total  |
+--------+----------+-----------------+
| L1Calo | 99.9012  | 7844422/7852181 |
| L2Calo |  0.6050  |  47506/7852181  |
|   L2   |  0.5601  |  43980/7852181  |
| EFCalo |  0.4207  |  33031/7852181  |
|  HLT   |  0.0019  |   149/7852181   |
+--------+----------+-----------------+


Filling...: 100%|██████████| 5/5 [00:12<00:00,  2.46s/it]

HLT_e140_lhloose_nod0_ringer_v13
+--------+----------+-----------------+
|  Step  | Eff [%%] |   passed/total  |
+--------+----------+-----------------+
| L1Calo | 99.9227  | 5629684/5634041 |
| L2Calo |  1.4273  |  80416/5634041  |
|   L2   |  1.3505  |  76088/5634041  |
| EFCalo |  1.1109  |  62586/5634041  |
|  HLT   |  0.0381  |   2149/5634041  |
+--------+----------+-----------------+


In [19]:
eff_data17_JF17.save()

In [20]:
del data17_df

## Emulate 2018:

In [21]:
data18_df = pd.concat( (load_hdf(dpath_data18), load_hdf(tpath_data18)), axis=1)

In [22]:
emulate(data18_df)

Emulate...: 100%|██████████| 4/4 [01:23<00:00, 20.77s/it]


In [23]:
data18_df.shape

(41873973, 112)

In [24]:
eff_data18_Zee = Efficiency( output_path+'/data18_Zee_efficiency_v13.root')

2022-03-24 10:04:19,473 | Py.Efficiency                           INFO Creating the StoreGate service with path: output/emulation/data18_Zee_efficiency_v13.root


In [25]:
eff_data18_Zee.fill( data18_df.loc[data18_df['target']==1], 'HLT_e17_lhvloose_nod0_ringer_v13_L1EM15VHI', pidname='el_lhvloose', reset=True)
eff_data18_Zee.fill( data18_df.loc[data18_df['target']==1], 'HLT_e28_lhtight_nod0_ringer_v13_ivarloose' , pidname='el_lhtight' , reset=True)
eff_data18_Zee.fill( data18_df.loc[data18_df['target']==1], 'HLT_e60_lhmedium_nod0_ringer_v13_L1EM24VHI', pidname='el_lhmedium', reset=True)
eff_data18_Zee.fill( data18_df.loc[data18_df['target']==1], 'HLT_e140_lhloose_nod0_ringer_v13'          , pidname='el_lhloose' , reset=True)

Filling...: 100%|██████████| 5/5 [01:54<00:00, 22.83s/it]


HLT_e17_lhvloose_nod0_ringer_v13_L1EM15VHI
+--------+----------+-------------------+
|  Step  | Eff [%%] |    passed/total   |
+--------+----------+-------------------+
| L1Calo | 94.1301  | 30705693/32620495 |
| L2Calo | 93.4850  | 30495282/32620495 |
|   L2   | 93.2561  | 30420616/32620495 |
| EFCalo | 91.9232  | 29985788/32620495 |
|  HLT   | 90.4972  | 29520633/32620495 |
+--------+----------+-------------------+


Filling...: 100%|██████████| 5/5 [01:23<00:00, 16.63s/it]


HLT_e28_lhtight_nod0_ringer_v13_ivarloose
+--------+----------+-------------------+
|  Step  | Eff [%%] |    passed/total   |
+--------+----------+-------------------+
| L1Calo | 94.6555  | 22067825/23313826 |
| L2Calo | 94.2085  | 21963610/23313826 |
|   L2   | 94.1132  | 21941383/23313826 |
| EFCalo | 93.3145  | 21755172/23313826 |
|  HLT   | 87.6071  | 20424565/23313826 |
+--------+----------+-------------------+


Filling...: 100%|██████████| 5/5 [00:05<00:00,  1.20s/it]


HLT_e60_lhmedium_nod0_ringer_v13_L1EM24VHI
+--------+----------+-----------------+
|  Step  | Eff [%%] |   passed/total  |
+--------+----------+-----------------+
| L1Calo | 99.5563  | 1375545/1381675 |
| L2Calo | 97.2738  | 1344008/1381675 |
|   L2   | 97.1878  | 1342819/1381675 |
| EFCalo | 95.0981  | 1313947/1381675 |
|  HLT   | 92.1110  | 1272675/1381675 |
+--------+----------+-----------------+


Filling...: 100%|██████████| 5/5 [00:00<00:00, 20.07it/s]

HLT_e140_lhloose_nod0_ringer_v13
+--------+----------+--------------+
|  Step  | Eff [%%] | passed/total |
+--------+----------+--------------+
| L1Calo | 99.9934  | 60385/60389  |
| L2Calo | 97.9549  | 59154/60389  |
|   L2   | 97.8589  | 59096/60389  |
| EFCalo | 96.6765  | 58382/60389  |
|  HLT   | 94.8666  | 57289/60389  |
+--------+----------+--------------+


In [26]:
eff_data18_Zee.save()

In [27]:
eff_data18_JF17 = Efficiency( output_path+'/data18_JF17_efficiency_v13.root')

2022-03-24 10:08:37,277 | Py.Efficiency                           INFO Creating the StoreGate service with path: output/emulation/data18_JF17_efficiency_v13.root


In [28]:
eff_data18_JF17.fill( data18_df.loc[data18_df['target']==0], 'HLT_e17_lhvloose_nod0_ringer_v13_L1EM15VHI', pidname='!el_lhvloose', reset=True)
eff_data18_JF17.fill( data18_df.loc[data18_df['target']==0], 'HLT_e28_lhtight_nod0_ringer_v13_ivarloose' , pidname='!el_lhvloose', reset=True)
eff_data18_JF17.fill( data18_df.loc[data18_df['target']==0], 'HLT_e60_lhmedium_nod0_ringer_v13_L1EM24VHI', pidname='!el_lhvloose', reset=True)
eff_data18_JF17.fill( data18_df.loc[data18_df['target']==0], 'HLT_e140_lhloose_nod0_ringer_v13'          , pidname='!el_lhvloose', reset=True)

Filling...: 100%|██████████| 5/5 [00:17<00:00,  3.54s/it]


HLT_e17_lhvloose_nod0_ringer_v13_L1EM15VHI
+--------+----------+-----------------+
|  Step  | Eff [%%] |   passed/total  |
+--------+----------+-----------------+
| L1Calo | 85.5183  | 7240922/8467098 |
| L2Calo |  2.0504  |  173611/8467098 |
|   L2   |  1.9913  |  168606/8467098 |
| EFCalo |  1.8195  |  154056/8467098 |
|  HLT   |  0.6164  |  52190/8467098  |
+--------+----------+-----------------+


Filling...: 100%|██████████| 5/5 [00:15<00:00,  3.19s/it]


HLT_e28_lhtight_nod0_ringer_v13_ivarloose
+--------+----------+-----------------+
|  Step  | Eff [%%] |   passed/total  |
+--------+----------+-----------------+
| L1Calo | 91.4008  | 6972786/7628804 |
| L2Calo |  0.9402  |  71728/7628804  |
|   L2   |  0.9052  |  69054/7628804  |
| EFCalo |  0.7956  |  60697/7628804  |
|  HLT   |  0.0422  |   3220/7628804  |
+--------+----------+-----------------+


Filling...: 100%|██████████| 5/5 [00:13<00:00,  2.74s/it]


HLT_e60_lhmedium_nod0_ringer_v13_L1EM24VHI
+--------+----------+-----------------+
|  Step  | Eff [%%] |   passed/total  |
+--------+----------+-----------------+
| L1Calo | 99.8995  | 6555389/6561985 |
| L2Calo |  0.9105  |  59748/6561985  |
|   L2   |  0.8689  |  57019/6561985  |
| EFCalo |  0.7367  |  48345/6561985  |
|  HLT   |  0.0509  |   3341/6561985  |
+--------+----------+-----------------+


Filling...: 100%|██████████| 5/5 [00:10<00:00,  2.07s/it]

HLT_e140_lhloose_nod0_ringer_v13
+--------+----------+-----------------+
|  Step  | Eff [%%] |   passed/total  |
+--------+----------+-----------------+
| L1Calo | 99.9159  | 4772502/4776518 |
| L2Calo |  1.9324  |  92303/4776518  |
|   L2   |  1.8537  |  88542/4776518  |
| EFCalo |  1.6233  |  77538/4776518  |
|  HLT   |  0.3303  |  15778/4776518  |
+--------+----------+-----------------+


In [29]:
eff_data18_JF17.save()